In [61]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [62]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [63]:
import pandas as pd
from transformers import BertTokenizerFast, TFBertForSequenceClassification
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import Sequential
import os
import numpy as np
import chardet
from sklearn.metrics import accuracy_score, precision_score, recall_score,confusion_matrix
import seaborn as sns
import re

In [64]:
dataframe = pd.read_csv("drive/MyDrive/Colab Notebooks/code smells tesi/Code_Smells_NLP-1/dataset/final_dataset.csv")


In [65]:
dataframe

,Component,CDSBP,CC,LC,LZC,RB,SC
0,/*\n * Licensed to the Apache Software Founda...,1,0,0,0,0,0
1,/*\n * Licensed to the Apache Software Founda...,0,0,0,0,0,1
2,/*\n * Licensed to the Apache Software Founda...,0,0,0,0,0,1
3,/*\n * Licensed to the Apache Software Founda...,0,0,0,0,0,1
4,/*\n * Licensed to the Apache Software Founda...,0,0,0,0,0,1
...,...,...,...,...,...,...,...
2512,"/*\n * The Apache Software License, Version 1....",0,0,0,0,0,1
2513,"/*\n * The Apache Software License, Version 1....",0,0,0,0,0,1
2514,"/*\n * The Apache Software License, Version 1....",0,0,0,0,0,1
2515,"/*\n * The Apache Software License, Version 1....",0,0,0,0,0,1


In [66]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

stop_words = set(stopwords.words('italian'))

def remove_comments(code):
    # Rimuove i commenti su una singola riga
    code = re.sub(r'//.*', '', code)
    # Rimuove i commenti su più righe
    code = re.sub(r'/\*.*?\*/', '', code, flags=re.DOTALL)
    # Rimuove le stopwords
    tokens = [word.lower() for word in code.split() if word.lower() not in stop_words]
    filtered_text = ' '.join(tokens)
    # Rimuoviamo gli import
    code = re.sub(r'import\s+.*?;', '', code, flags=re.DOTALL)
    # Rimuoviamo i package
    code = re.sub(r'package\s+.*?;', '', code, flags=re.DOTALL)
    #Rimuove \n e \t
    code = re.sub(r'[\n\t]', '', code)
    return code

for i in range(len(dataframe)):
    dataframe['Component'][i] = remove_comments(dataframe['Component'][i])

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
<ipython-input-66-4d4971366606>:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe['Component'][i] = remove_comments(dataframe['Component'][i])


In [67]:
# Carica il tokenizer
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')

# Carica il modello
model = TFBertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=6)


All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [68]:
# Creazione X e y. X = dataset senza le etichette, y = etichette
dataframe2 = dataframe.drop(columns=['CDSBP','CC','LC','LZC','RB','SC'])
y = dataframe[['CDSBP','CC','LC','LZC','RB','SC']]

In [69]:
# Divisione in train, validation e test set
# X_train, X_test, Y_train, Y_test = train_test_split(dataframe2, y, test_size=0.33, random_state=1)
X_train, X_val_test, Y_train, Y_val_test = train_test_split(dataframe2, y, test_size=0.4, random_state=1)
X_val, X_test, Y_val, Y_test = train_test_split(X_val_test, Y_val_test, test_size=0.5, random_state=1)

In [70]:
l1 = [str(i) for i in X_train['Component'].tolist()]
l2 = [str(i) for i in X_val['Component'].tolist()]
l3 = [str(i) for i in X_test['Component'].tolist()]

In [71]:
# max_len = 256

# def split_string(s, chunk_size):
#     return [s[i:i+chunk_size] for i in range(0, len(s), chunk_size)]

# def tokenize_and_join(tensor_list):
#     return tf.concat(tensor_list, axis=0)

# def tokenize_class(class_string):
#     chunks = split_string(class_string, max_len)
#     token_chunks = [tokenizer.tokenize(chunk, padding=True, truncation=True, max_length=256, return_tensors='tf') for chunk in chunks]
#     tokens = tokenize_and_join(token_chunks)
#     return tf.constant(tokens)

# class_tensors = []
# for i in range(len(l1)):
#     class_tensor = tokenize_class(l1[i])
#     class_tensors.append(class_tensor)
# print(type(class_tensors))
# print(len(class_tensors))


In [72]:
# max_len = 256

# def split_string(s, chunk_size):
#     return [s[i:i+chunk_size] for i in range(0, len(s), chunk_size)]

# def tokenize_and_join(tensor_list):
#     return tf.concat(tensor_list, axis=0)

# def tokenize_class(class_string):
#     chunks = split_string(class_string, max_len)
#     token_chunks = [tokenizer.tokenize(chunk, padding=True, truncation=True, max_length=256, return_tensors='tf') for chunk in chunks]
#     tokens = tokenize_and_join(token_chunks)
#     return tf.constant(tokens)

# class_tensors_val = []
# for i in range(len(l2)):
#     class_tensor = tokenize_class(l2[i])
#     class_tensors_val.append(class_tensor)
# print(type(class_tensors_val))
# print(len(class_tensors_val))


In [73]:
X_train_tokenized = tokenizer(l1, padding=True, truncation=True, max_length=256, return_tensors='tf')
print("FATTA TOKENIZZAZIONE")

FATTA TOKENIZZAZIONE


In [74]:
X_val_tokenized = tokenizer(l2, padding=True, truncation=True, max_length=256, return_tensors='tf')
print("FATTA TOKENIZZAZIONE")

FATTA TOKENIZZAZIONE


In [75]:
X_test_tokenized = tokenizer(l3, padding=True, truncation=True, max_length=256, return_tensors='tf')
print("FATTA TOKENIZZAZIONE")

FATTA TOKENIZZAZIONE


In [76]:
# X_train_input_ids = tf.constant([], dtype=tf.int32)
# for i in range(len(class_tensors)):
#     input_ids = tf.strings.to_number(class_tensors[i][0], out_type=tf.int32)
#     X_train_input_ids = tf.concat([X_train_input_ids, input_ids], axis=0)
# X_train_attention_mask = class_tensors[0][1].numpy()

# X_val_input_ids = tf.strings.to_number(class_tensors_val[0][0], out_type=tf.int32).numpy()
# X_val_attention_mask = class_tensors_val[0][1].numpy()

# X_test_input_ids = X_test_tokenized['input_ids']
# X_test_attention_mask = X_test_tokenized['attention_mask']
# print(type(X_test_input_ids))


In [77]:
# Estrai le sequenze di token e le maschere di attenzione
X_train_input_ids = X_train_tokenized['input_ids']
X_train_attention_mask = X_train_tokenized['attention_mask']

X_val_input_ids = X_val_tokenized['input_ids']
X_val_attention_mask = X_val_tokenized['attention_mask']

X_test_input_ids = X_test_tokenized['input_ids']
X_test_attention_mask = X_test_tokenized['attention_mask']

In [78]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [79]:
history = model.fit(
    x=X_train_input_ids,
    y=Y_train,
    batch_size=32,
    epochs=20,
    validation_data=(X_val_input_ids, Y_val),
    verbose=1,
    callbacks=[EarlyStopping(patience=6)],
    )

Epoch 1/20
48/48 [==============================] - 139s 2s/step - loss: 2.3895 - accuracy: 0.3013 - val_loss: 2.5708 - val_accuracy: 0.3638
Epoch 2/20
48/48 [==============================] - 84s 2s/step - loss: 2.5708 - accuracy: 0.3974 - val_loss: 2.5708 - val_accuracy: 0.3638
Epoch 3/20
48/48 [==============================] - 86s 2s/step - loss: 2.5708 - accuracy: 0.3974 - val_loss: 2.5708 - val_accuracy: 0.3638
Epoch 4/20
48/48 [==============================] - 86s 2s/step - loss: 2.5708 - accuracy: 0.3974 - val_loss: 2.5708 - val_accuracy: 0.3638
Epoch 5/20
48/48 [==============================] - 86s 2s/step - loss: 2.5708 - accuracy: 0.3974 - val_loss: 2.5708 - val_accuracy: 0.3638
Epoch 6/20
48/48 [==============================] - 86s 2s/step - loss: 2.5708 - accuracy: 0.3974 - val_loss: 2.5708 - val_accuracy: 0.3638
Epoch 7/20
48/48 [==============================] - 86s 2s/step - loss: 2.5708 - accuracy: 0.3974 - val_loss: 2.5708 - val_accuracy: 0.3638


In [80]:
# Fai le predizioni sul test set

Y_pred = model.predict([X_test_input_ids,X_test_attention_mask])

16/16 [==============================] - 14s 568ms/step


In [81]:
# Calcola l'accuracy
Y_test_binary = np.argmax(Y_test.values, axis=1)
accuracy = accuracy_score(Y_test_binary, Y_pred.logits.argmax(axis=1))
precision = precision_score(Y_test_binary, Y_pred.logits.argmax(axis=1), average="macro")
recall = recall_score(Y_test_binary, Y_pred.logits.argmax(axis=1), average="macro")
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)

Accuracy: 0.38095238095238093
Precision: 0.06349206349206349
Recall: 0.16666666666666666


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [82]:
# Calcola la matrice di confusione
cm = confusion_matrix(Y_test_binary, Y_pred.logits.argmax(axis=1))

# Stampa la matrice di confusione
print(cm)


[[  0   0   0   0   0 113]
 [  0   0   0   0   0  36]
 [  0   0   0   0   0  47]
 [  0   0   0   0   0  33]
 [  0   0   0   0   0  83]
 [  0   0   0   0   0 192]]


In [83]:
# # Aggiungi un layer di output con funzione di attivazione softmax

# # Crea un nuovo modello Keras sequenziale
# new_model = Sequential()

# # Aggiungi il modello BERT come primo layer
# new_model.add(model)

# # Aggiungi il tuo nuovo layer come secondo layer
# new_model.add(Dense(6, activation='softmax'))

# # Ora puoi utilizzare il nuovo modello per addestrare e valutare il tuo task specifico
# new_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])


In [84]:
# history = new_model.fit(
#     x=X_train_input_ids,
#     y=Y_train,
#     batch_size=32,
#     epochs=10,
#     validation_data=(X_val_input_ids, Y_val),
#     verbose=1,
#     callbacks=[EarlyStopping(patience=3)],
#     )

In [85]:
# new_model = Sequential()
# new_model.add(model)
# new_model.add(Dense(64, activation='softmax'))
# new_model.add(Dense(6, activation='sigmoid'))


In [86]:
# from transformers import BertTokenizer
# tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [87]:
# X_train_tokenized = tokenizer(l1, padding=True, truncation=True, max_length=256, return_tensors='tf')
# X_val_tokenized = tokenizer(l2, padding=True, truncation=True, max_length=256, return_tensors='tf')
# X_test_tokenized = tokenizer(l3, padding=True, truncation=True, max_length=256, return_tensors='tf')
# print("FATTA TOKENIZZAZIONE")

In [88]:
# # Estrai le sequenze di token e le maschere di attenzione
# X_train_input_ids = X_train_tokenized['input_ids']
# X_train_attention_mask = X_train_tokenized['attention_mask']

# X_val_input_ids = X_val_tokenized['input_ids']
# X_val_attention_mask = X_val_tokenized['attention_mask']

# X_test_input_ids = X_test_tokenized['input_ids']
# X_test_attention_mask = X_test_tokenized['attention_mask']

In [89]:
# new_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
# new_history = new_model.fit(
#     x=X_train_input_ids,
#     y=Y_train,
#     batch_size=32,
#     epochs=10,
#     validation_data=(X_val_input_ids, Y_val),
#     verbose=1,
#     callbacks=[EarlyStopping(patience=6)],
# )


In [90]:
# new_Y_pred = new_model.predict([X_test_input_ids,X_test_attention_mask])
# new_Y_test_binary = np.argmax(Y_test.values, axis=1)
# new_accuracy = accuracy_score(new_Y_test_binary, new_Y_pred.argmax(axis=1))
# print("New accuracy:", new_accuracy)
